<a href="https://colab.research.google.com/github/tardigrade-dot/colab-script/blob/main/generate_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!which python

In [ ]:
!echo $HOSTNAME

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

安装sensevoice环境

In [ ]:
!pip install -r https://raw.githubusercontent.com/FunAudioLLM/SenseVoice/refs/heads/main/requirements.txt

In [ ]:
!pip install wetext

In [ ]:
# sv311 安装SenseVoice环境: https://github.com/FunAudioLLM/SenseVoice

import re
import os
from pathlib import Path
import string
from funasr import AutoModel
import difflib
from pydub import AudioSegment
# from zh_normalization import TextNormalizer
from wetext import Normalizer

class CtcForcdAlign:
    def __init__(self, model_dir, device) -> None:

        self.MIN_CHARS_PER_LINE = 10
        self.model = AutoModel(
            model=model_dir,
            vad_model="fsmn-vad",
            vad_kwargs={"max_single_segment_time": 30_000},
            device=device,
            disable_update=True,
            trust_remote_code=True,
            # remote_code="./extro/model.py",
            # remote_code="funasr.models.sense_voice.model",
        )

        self.normalizer = Normalizer(lang="zh", operator="tn", remove_erhua=True, traditional_to_simple=True)
        chinese_pattern = r"（.*?）"
        english_pattern = r"\([^)]*?\)"

        self.combined_pattern = f"{chinese_pattern}|{english_pattern}"
        self.PUNCTUATION_CHARS = set(string.punctuation) | set(',.?，。！？；：、“”‘’《》【】（）')
        self.PUNCT_REGEX = re.compile(r'[.,:;!?，。、；：？！“‘”’（）《》【】\s-]')

    def is_punctuation(self, token):
        return token in self.PUNCTUATION_CHARS

    def map_opcodes_to_raw(self, clean_opcodes, asr_timestamps, correct_tokens_raw):
        aligned_timestamps = []
        status = []

        j_raw_ptr = 0

        for tag, i1, i2, j1, j2 in clean_opcodes:

            while j_raw_ptr < len(correct_tokens_raw) and self.PUNCT_REGEX.match(correct_tokens_raw[j_raw_ptr]):
                aligned_timestamps.append((None, None))
                j_raw_ptr += 1

            asr_chars_consumed = i2 - i1
            if tag == "equal":
                for k in range(j2 - j1): # 遍历匹配的汉字
                    aligned_timestamps.append(asr_timestamps[i1 + k])
                    status.append(True)
                    j_raw_ptr += 1 # 原始指针移动一个汉字
                    while j_raw_ptr < len(correct_tokens_raw) and self.PUNCT_REGEX.match(correct_tokens_raw[j_raw_ptr]):
                        aligned_timestamps.append((None, None))
                        j_raw_ptr += 1
            elif tag == "replace" or tag == "insert":
                for k in range(j2 - j1): # 遍历差异的汉字
                    if asr_chars_consumed > 0:
                        idx = i1 + k % asr_chars_consumed
                        aligned_timestamps.append(asr_timestamps[idx])
                    else: # 纯粹的插入 (tag='insert')
                        aligned_timestamps.append((None, None))
                    status.append(False)
                    j_raw_ptr += 1 # 原始指针移动一个汉字
                    while j_raw_ptr < len(correct_tokens_raw) and self.PUNCT_REGEX.match(correct_tokens_raw[j_raw_ptr]):
                        aligned_timestamps.append((None, None))
                        j_raw_ptr += 1
            elif tag == "delete":
                continue

        while j_raw_ptr < len(correct_tokens_raw) and self.PUNCT_REGEX.match(correct_tokens_raw[j_raw_ptr]):
            aligned_timestamps.append((None, None))
            j_raw_ptr += 1
        if len(aligned_timestamps) == len(correct_tokens_raw):
            return aligned_timestamps, status
        else:
            print(f"🚨 警告: 最终长度不匹配. Raw: {len(correct_tokens_raw)}, Aligned: {len(aligned_timestamps)}")
            return aligned_timestamps, status

    def map_asr_to_correct(self, asr_tokens, asr_timestamps, correct_tokens):

        punctuation_regex = r'[.,:;!?，。、；：？！“‘”’（）《》【】\s-]'
        clean_correct_tokens = re.sub(punctuation_regex, '', correct_tokens)
        matcher_clean = difflib.SequenceMatcher(None, asr_tokens, clean_correct_tokens)
        clean_opcodes = matcher_clean.get_opcodes()

        return self.map_opcodes_to_raw(clean_opcodes, asr_timestamps, correct_tokens)

    #old version
    def map_asr_to_correct_oldversion(self, asr_tokens, asr_timestamps, correct_tokens):
        """
        将ASR token时间戳映射到正确文本 token
        返回：
            aligned_timestamps: 对齐后的正确文本时间戳列表
            status: 每个 token 是否匹配
        """
        aligned_timestamps = []
        status = []

        # asr_tokens = "".join(asr_tokens)
        # 使用 difflib 对齐 ASR 与正确文本
        matcher = difflib.SequenceMatcher(self.is_punctuation, asr_tokens, correct_tokens)
        for tag, i1, i2, j1, j2 in matcher.get_opcodes():
            if tag == "equal":
                # token 匹配 → 直接继承时间戳
                for k in range(j2 - j1):
                    aligned_timestamps.append(asr_timestamps[i1 + k])
                    status.append(True)
            elif tag == "replace":
                # token 替换 → 尝试平均分配时间戳
                # 可以选择标记为不匹配
                for k in range(j2 - j1):
                    if (i2 - i1) > 0:
                        idx = i1 + k % (i2 - i1)
                        aligned_timestamps.append(asr_timestamps[idx])
                    else:
                        # ASR 没有对应 token → 填充 None
                        aligned_timestamps.append((None, None))
                    status.append(False)
            elif tag == "insert":
                # 正确文本中新增 token → 无对应 ASR token
                for _ in range(j2 - j1):
                    aligned_timestamps.append((None, None))
                    status.append(False)
            elif tag == "delete":
                # ASR 中多余 token → 忽略
                continue

        return aligned_timestamps, status

    def is_valid_time(self, t):
        return isinstance(t, (int, float)) and t >= 0

    def get_valid_start_end(self, words):
        start_time, end_time = None, None

        for w in words:
            t = w.get('start')
            if self.is_valid_time(t):
                start_time = t
                break

        for w in reversed(words):
            t = w.get('end')
            if self.is_valid_time(t):
                end_time = t
                break

        return start_time, end_time

    def ms_to_srt_time_format(self, ms):
        # 确保输入是整数或可以转换为整数
        ms = int(ms)
        # 计算小时、分钟、秒和毫秒
        hours = ms // 3600000
        minutes = (ms % 3600000) // 60000
        seconds = (ms % 60000) // 1000
        milliseconds = ms % 1000

        # 使用 f-string 格式化输出，确保每个部分都有固定的位数
        return f"{hours:02d}:{minutes:02d}:{seconds:02d},{milliseconds:03d}"

    def generate_srt_from_words_and_timestamps(self, words, timestamps):
        if not words or not timestamps or len(words) != len(timestamps):
            print(f"words length[{len(words)}] not equal to timestamps length[{len(timestamps)}]")
            raise Exception('process error')

        srt_content = ""
        subtitle_index = 1
        sentence_delimiters = ['。', '？', '！', '.', '?', '!', '…']

        current_subtitle_words = []
        end_time = 0

        # 内部函数：负责判断是否满足长度要求并输出字幕
        def _write_subtitle(words_to_write, pre_end_time):
            nonlocal srt_content, subtitle_index

            if not words_to_write:
                return None

            # start_time = words_to_write[0]['start']
            # end_time = words_to_write[-1]['end']
            start_time, end_time = self.get_valid_start_end(words_to_write)
            if pre_end_time and pre_end_time != 0 and start_time < pre_end_time:
                start_time = pre_end_time

            # 将单词列表组合成一个完整的句子
            sentence_text = "".join([w['word'] for w in words_to_write])

            srt_content += str(subtitle_index) + "\n"
            srt_content += f"{self.ms_to_srt_time_format(start_time)} --> {self.ms_to_srt_time_format(end_time)}\n"
            srt_content += sentence_text + "\n\n"
            subtitle_index += 1
            return end_time

        for i in range(len(words)):
            word = words[i]
            timestamp = timestamps[i]

            current_subtitle_words.append({'word': word, 'start': timestamp[0], 'end': timestamp[1]})

            is_sentence_end = word in sentence_delimiters
            if is_sentence_end:
                current_text = "".join([w['word'] for w in current_subtitle_words])
                current_length = len(current_text)
                if current_length >= self.MIN_CHARS_PER_LINE:
                    end_time = _write_subtitle(current_subtitle_words, end_time)
                    current_subtitle_words = [] # 清空累计列表
        if current_subtitle_words:
            all_none = all(
                item['start'] is None and item['end'] is None
                for item in current_subtitle_words
            )
            if not all_none:
                _write_subtitle(current_subtitle_words, end_time)

        return srt_content

    def generate_srt_file(self, wav_file, over_write=False):

        wav_path = Path(wav_file)

        txt_path = wav_path.with_suffix('.txt')
        srt_path = wav_path.with_suffix('.srt')

        txt_file = str(txt_path)
        srt_file = str(srt_path)

        print(f'\nwav [{wav_file}], \ntxt [{txt_file}], \nsrt [{srt_file}]')

        if not wav_path.exists():
            raise Exception(f'wav file[{wav_file}] not exists ')
        if not txt_path.exists():
            raise Exception(f'txt file[{txt_file}] not exists ')
        if srt_path.exists() and not over_write:
            print(f'srt file[{srt_file}] exists, just return ')
            return

        with open(txt_file, 'r') as f:
            target_txt = "".join(f.readlines())

        res = self.model.generate(
                input=wav_file,
                cache={},
                language="auto",  # "zn", "en", "yue", "ja", "ko", "nospeech"
                use_itn=False,
                batch_size_s=30,
                merge_vad=True,
                merge_length_s=20,
                output_timestamp=True,
            )
        output = res[0]

        target_txt = re.sub(self.combined_pattern, "", target_txt)
        target_txt = target_txt.replace("\n", " ")
        target_txt = self.normalizer.normalize(target_txt)

        aligned_timestamps, status = self.map_asr_to_correct(output["words"], output["timestamp"], target_txt)

        num_tokens = len(status)
        num_mismatch = status.count(False)
        mismatch_ratio = num_mismatch / num_tokens

        if mismatch_ratio > 0.05:
            print(f"⚠️❌ 注意, 不匹配率过高.  不匹配 token({num_mismatch}/{num_tokens}) 占比: {mismatch_ratio:.2%}")
        else:
            print(f"⚠️ 不匹配 token({num_mismatch}/{num_tokens}) 占比: {mismatch_ratio:.2%}")

        srt_result = self.generate_srt_from_words_and_timestamps(target_txt, aligned_timestamps)

        with open(srt_file, 'w') as f:
            f.write(srt_result)

        print(f"字幕已生成, 保存在:{srt_file}")

    def get_wav_list_sorted(self, wav_src_dir, wav_regex):
        pattern = re.compile(wav_regex)

        files = [
            f for f in os.listdir(wav_src_dir)
            if f.endswith('.wav') and pattern.match(f)
        ]

        def sort_key(filename):
            m = pattern.match(filename)
            if m:
                second, first = map(int, m.groups())
                return (second, first)
            return (float('inf'), float('inf'))

        sorted_files = sorted(files, key=sort_key)

        sorted_paths = [os.path.join(wav_src_dir, f) for f in sorted_files]
        return sorted_paths

    def generate_srt_dir(self, wav_src_dir, wav_regex, over_write=False):
        for w_path in self.get_wav_list_sorted(wav_src_dir, wav_regex):
            self.generate_srt_file(w_path, over_write)

    def asr_with_target(self, wav_file):

        wav_path = Path(wav_file)
        txt_path = wav_path.with_suffix('.txt')

        output_dir = "/Volumes/sw/datasets_prepare/meiguodegushi_output/" + wav_path.stem
        if os.path.exists(output_dir):

            print(f'⚠️ output dir exists {output_dir}, nothing will process ...')
            return
        target = []
        with open(str(txt_path), 'r') as f:
            target = f.read().splitlines()

        res = self.model.generate(
            input=wav_file,
            cache={},
            language="auto",  # "zn", "en", "yue", "ja", "ko", "nospeech"
            use_itn=False,
            batch_size_s=30,
            merge_vad=True,
            merge_length_s=20,
            output_timestamp=True,
        )
        output = res[0]
        sent_ts = output["timestamp_sentence"]

        target_txt = "".join(target)
        target_txt = self.normalizer.normalize(target_txt)

        target_txt = re.sub(self.combined_pattern, "", target_txt)
        target_txt = target_txt.replace("\n", "")

        target_tokens = self.model.kwargs['tokenizer'].text2tokens(target_txt)
        aligned_timestamps, status = self.map_asr_to_correct(output["words"], output["timestamp"], target_tokens)
        self.check_status(status)
        sen_clips = self.split_tokens_by_sentence(sent_ts, aligned_timestamps, target_tokens)
        self.export_audio_segments(wav_file, sen_clips, output_dir)

        print(f'✅ fininsh process wav {wav_file}')

    def export_audio_segments(self, audio_file, segments, output_dir):
        """
        audio_file: 原始音频路径
        segments: [{'text':..., 'start':..., 'end':...}, ...]
            start/end 单位：毫秒
        output_dir: 输出目录
        """
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        audio = AudioSegment.from_file(audio_file)

        for i, seg in enumerate(segments, 1):
            start_ms = seg['start']
            end_ms = seg['end']
            clip = audio[start_ms:end_ms]
            # 保存 wav
            wav_path = os.path.join(output_dir, f"{i:03d}.wav")
            clip.export(wav_path, format="wav")

            # 保存对应文本
            txt_path = os.path.join(output_dir, f"{i:03d}.txt")
            with open(txt_path, "w", encoding="utf-8") as f:
                f.write(seg['text'])

            # print(f"导出: {wav_path}, {txt_path}")

    def merge_short_sentences(self, sentences, min_duration=3000):
        """
        sentences: [{'text':..., 'start':..., 'end':...}, ...]
        min_duration: 最小时间阈值，单位与 start/end 一致（示例中为毫秒）
        """
        if not sentences:
            return []

        merged = [sentences[0]]

        for s in sentences[1:]:
            duration = s['end'] - s['start']
            if duration < min_duration:
                # 合并到上一条
                prev = merged[-1]
                prev['text'] += s['text']
                prev['end'] = s['end']
            else:
                merged.append(s)

        return merged

    def merge_sent_timestamps(self, sent_ts):
        """将每句中 token 的时间范围合并为 (start, end)"""
        if len(sent_ts) % 2 != 0:
            raise Exception('sent ts count not match with start and end timestamp')
        merged = []
        for i in range(0, len(sent_ts), 2):
            pair = sent_ts[i:i+2]  # 取两元素
            start = pair[0][0]
            end = pair[-1][1]
            merged.append((start, end))
        return merged

    def split_tokens_by_sentence(self, sent_ts, aligned_timestamps, target_tokens):
        assert len(aligned_timestamps) == len(target_tokens), "时间戳与token数量不一致"
        results = []
        idx = 0
        n = len(target_tokens)
        for sent_start, sent_end in sent_ts:
            sent_tokens = []
            # 遍历 token，直到 token 起点超过句子结束
            while idx < n:
                token_start, token_end = aligned_timestamps[idx]
                if token_start is not None and token_end is not None:
                    # token 完全在句子区间内
                    if token_start >= sent_start and token_end <= sent_end:
                        sent_tokens.append(target_tokens[idx])
                        idx += 1
                    # token 超出当前句子区间，说明当前句子结束
                    elif token_start >= sent_end or token_end <= sent_start:
                        break
                else:
                    sent_tokens.append(target_tokens[idx])
                    idx += 1

            if sent_tokens:
                sentence = "".join(sent_tokens)
                results.append({
                    "text": sentence,
                    "start": sent_start,
                    "end": sent_end
                })

        return results

    def check_status(self, status):
        num_tokens = len(status)
        num_mismatch = status.count(False)
        mismatch_ratio = num_mismatch / num_tokens

        if mismatch_ratio > 0.3:
            print(f"❌❌ 注意, 不匹配率过高.  不匹配 token({num_mismatch}/{num_tokens}) 占比: {mismatch_ratio:.2%}")
            raise Exception()
        if mismatch_ratio > 0.15:
            print(f"⚠️❌ 注意, 不匹配率过高.  不匹配 token({num_mismatch}/{num_tokens}) 占比: {mismatch_ratio:.2%}")
        else:
            print(f"⚠️ 不匹配 token({num_mismatch}/{num_tokens}) 占比: {mismatch_ratio:.2%}")

    def create_clip_with_asr(self, audio_dir):
        audio_files = sorted(os.listdir(audio_dir))
        for _a_f in audio_files:

            if _a_f.endswith('.wav'):

                _audio_file = os.path.join(audio_dir, _a_f)

                res_dir =Path(_audio_file).with_suffix("")
                res_dir.mkdir(parents=True, exist_ok=True)

                res = self.model.generate(
                    input=_audio_file,
                    cache={},
                    language="auto",  # "zn", "en", "yue", "ja", "ko", "nospeech"
                    use_itn=False,
                    batch_size_s=30,
                    merge_vad=True,
                    merge_length_s=20,
                    output_timestamp=True,
                )
                output = res[0]

                audio = AudioSegment.from_file(_audio_file)
                sent_timestamps = output['timestamp_sentence']
                words = output['words']
                words_ts = output['timestamp']

                for _i, sent_ts in enumerate(sent_timestamps):
                    _start_ts = sent_ts[0]
                    _end_ts = sent_ts[1]
                    clip = audio[_start_ts: _end_ts]

                    _clip_txt = []
                    for w, (w_s_ts, w_e_ts) in zip(words, words_ts):
                        if w_s_ts >= _start_ts and w_e_ts <= _end_ts:
                            _clip_txt.append(w)
                        if w_s_ts > _end_ts:
                            break
                    clip_txt = "".join(_clip_txt)

                    clip_path = res_dir / f"{_i:03d}.txt"
                    clip_path.write_text(clip_txt, encoding="utf-8")

                    wav_path = os.path.join(res_dir, f"{_i:03d}.wav")
                    clip.export(wav_path, format="wav")

                print(f'✅ ✅ ✅  文件{_a_f}处理完成')

    def check_audio(self, audio_dir):

        sub_data_dir_list = sorted(os.listdir(audio_dir))

        for _i, _d in enumerate(sub_data_dir_list):
            _d2 = os.path.join(audio_dir, _d)
            if Path(_d2).is_dir():

                for _f in sorted(os.listdir(_d2)):
                    if _f.endswith('.wav'):

                        _audio_file = os.path.join(_d2, _f)
                        txt_file = Path(_audio_file).with_suffix('.txt')
                        file_text = Path(txt_file).read_text()

                        res = self.model.generate(
                            input=_audio_file,
                            cache={},
                            language="auto",  # "zn", "en", "yue", "ja", "ko", "nospeech"
                            use_itn=False,
                            batch_size_s=30,
                            merge_vad=True,
                            merge_length_s=20,
                        )
                        text = res[0]['text']
                        asr_text = re.sub(r"(?:<\|[^|>]+\|>)+", "", text).replace(" ", "")

                        file_text = re.sub(r"[^\u4e00-\u9fa5a-zA-Z0-9]", "", file_text).replace(" ", "")

                        matcher = difflib.SequenceMatcher(None, asr_text, file_text)
                        similarity = matcher.ratio()
                        error_rate = 1 - similarity

                        print(f"相似度: {similarity:.4f}, 错误率: {error_rate:.4f}")
                        if error_rate < 0.1 and len(file_text) == len(asr_text):
                            continue
                        if error_rate > 0.09 :
                            print(f'⚠️ ⚠️ ⚠️ {_audio_file}')
                            print(f'⚠️ ⚠️ ⚠️ 错误率 {error_rate}')
                            print(f'asr_text:{asr_text}')
                            print(f'file_text:{file_text}')
                            print('')
                print(f'✅ ✅ ✅ [{_i}]完成处理路径:{_d2}')


env = "colab" # colab local
if env == "colab":
    cfa = CtcForcdAlign(
        # model_dir = "/Volumes/sw/pretrained_models/SenseVoiceSmall",
        model_dir = "iic/SenseVoiceSmall",
        device = "cuda"
    )
    cfa.generate_srt_dir("/content/drive/MyDrive/tianchaoyaoyuan1", r"tianchaoyaoyuan1-(\d+)_(\d+)\.wav")
elif env == 'local':
    cfa = CtcForcdAlign(
        model_dir = "/Users/larry/github.com/tardigrade-dot/SenseVoice/models/SenseVoiceSmall",
        device = "cpu"
    )
    cfa.generate_srt_dir("/Users/larry/Documents/epub/fubaiyufanfu", r"fubaiyufanfu-(\d+)_(\d+)\.wav")
else:
    raise Exception('env not support ')
# cfa.generate_srt_dir('/Volumes/sw/MyDrive/zhengzhi1/output', r"zhengzhi1-(\d+)_(\d+)\.wav", True)
# cfa.generate_srt_file("/Users/larry/github.com/tardigrade-dot/SenseVoice/data-forcedaligner/data.wav", True)

# wav_dir = "/Volumes/sw/datasets_prepare/zhongdong"
# for wav_file in os.listdir(wav_dir):
#     if wav_file.endswith('.wav'):
#         wav_path = os.path.join(wav_dir, wav_file)
#         cfa.asr_with_target(wav_path)

# cfa.asr_with_target("/Volumes/sw/datasets_prepare/meiguodegushi/真人朗读有声书《美国的故事》从1517宗教改革到美国建国400多年的风风雨雨 p05 05.美洲大陆的主人.mp3")
# cfa.generate_srt_file("/Volumes/sw/MyDrive/zhengzhi1/output/zhengzhi1-1_3.wav", True)

# cfa.check_audio("/Volumes/sw/datasets_prepare/meiguodegushi_output")

# cfa.create_clip_with_asr("/Volumes/sw/datasets_prepare/qiangpao")

# cfa.generate_srt_dir("/Users/larry/Documents/epub/quanliyuwuzhi", r"quanliyuwuzhi-(\d+)_(\d+)\.wav",)

# cfa.generate_srt_dir("/content/drive/MyDrive/tianchaoyaoyuan1", r"tianchaoyaoyuan1-(\d+)_(\d+)\.wav")

# cfa.generate_srt_file("/Users/larry/github.com/tardigrade-dot/colab-script2/data_src/zhipei-0_0.wav")


生成视频

In [ ]:
!pip install moviepy==2.2.1 pysrt tqdm

In [ ]:
!ffmpeg -encoders | grep 'nvenc\|cuda'

In [ ]:
# torch311
# /Users/larry/miniconda3/envs/torch311/bin/python /Users/larry/github.com/tardigrade-dot/colab-script/moviepy_example.py
from moviepy import *
# from tqdm import tqdm
from tqdm.notebook import tqdm
import pysrt
import os
from pathlib import Path
import re
import subprocess
import shlex

# ---- 字体路径 ----
FONT_PATH = "/content/drive/MyDrive/data_src/pingfang-sc-regular.ttf"

# ----------------------------
# 编码器 / CUDA 检测 与 配置（只影响编码参数）
# ----------------------------
# 可选："auto" / "fast" / "quality" / "cpu"
# "auto"：如果检测到 NVENC 则用 fast 或 quality 中之一（由 ENCODER_MODE 结合下面的选择逻辑决定）
# "fast"：使用速度优先的 NVENC 配置（若无 NVENC 回退到 libx264）
# "quality"：使用画质优先的 NVENC 配置（若无 NVENC 回退到 libx264）
# "cpu"：强制使用 libx264（CPU）
ENCODER_MODE = "cpu"  # 可改为 "fast" / "quality" / "cpu"

def detect_nvenc():
    """
    检查本机 ffmpeg 是否支持 h264_nvenc 编码器。
    返回 True（支持）或 False（不支持或 ffmpeg 不可用）。
    """
    try:
        # 运行 ffmpeg -hide_banner -encoders 并检查输出
        p = subprocess.run(["ffmpeg", "-hide_banner", "-encoders"], capture_output=True, text=True, timeout=8)
        out = p.stdout + p.stderr
        return "h264_nvenc" in out
    except Exception:
        return False

_NVENC_AVAILABLE = detect_nvenc()

def get_encoder_config(mode=ENCODER_MODE):
    """
    返回三元组:
      - codec_for_moviepy (string)  e.g. "h264_nvenc" or "libx264"
      - ffmpeg_params_for_moviepy (list)  (传给 moviepy.write_videofile 的 ffmpeg_params)
      - extra_concat_params (list)  (在 combine_mp4 subprocess 调用中插入，例: ['-c:v','h264_nvenc','-preset','p4', ...])
    仅修改编码相关参数，其他逻辑不变。
    """
    # CPU fallback params (libx264)
    cpu_moviepy_params = ["-movflags", "+faststart", "-preset", "medium"]
    cpu_concat_params = ["-c:v", "libx264", "-preset", "medium"]

    # NVENC 快速（速度优先） - 移除不兼容的 -rc 和 -cq
    fast_nvenc_moviepy = ["-movflags", "+faststart", "-preset", "p3"]
    fast_nvenc_concat = ["-c:v", "nvenc", "-preset", "p3"] # <<< 更改为 'nvenc'

    # NVENC 画质优先（更高质量） - 移除不兼容的 -rc 和 -cq
    quality_nvenc_moviepy = ["-movflags", "+faststart", "-preset", "p6"]
    quality_nvenc_concat = ["-c:v", "h264_nvenc", "-preset", "p6"]

    if mode == "cpu":
        return "libx264", cpu_moviepy_params, cpu_concat_params

    if mode == "fast":
        if _NVENC_AVAILABLE:
            return "h264_nvenc", fast_nvenc_moviepy, fast_nvenc_concat
        else:
            return "libx264", cpu_moviepy_params, cpu_concat_params

    if mode == "quality":
        if _NVENC_AVAILABLE:
            return "h264_nvenc", quality_nvenc_moviepy, quality_nvenc_concat
        else:
            return "libx264", cpu_moviepy_params, cpu_concat_params

    # mode == "auto"
    if _NVENC_AVAILABLE:
        # 默认选择 fast NVENC 当 auto
        return "h264_nvenc", fast_nvenc_moviepy, fast_nvenc_concat
    else:
        return "libx264", cpu_moviepy_params, cpu_concat_params

# # get config once
# _CODEC_FOR_MOVIEPY, _FFMPEG_PARAMS_FOR_MOVIEPY, _CONCAT_PARAMS = get_encoder_config(ENCODER_MODE)
# # 示例：将编码器名称从 'h264_nvenc' 切换到 'nvenc'
# _CODEC_FOR_MOVIEPY = "nvenc"
# _FFMPEG_PARAMS_FOR_MOVIEPY = ["-movflags", "+faststart", "-preset", "p3"]
# _CONCAT_PARAMS = ["-c:v", _CODEC_FOR_MOVIEPY, "-preset", "p3"]

_CODEC_FOR_MOVIEPY, _FFMPEG_PARAMS_FOR_MOVIEPY, _CONCAT_PARAMS = get_encoder_config(ENCODER_MODE)
# ----------------------------
# 以上部分仅涉及编码/硬件检测（不改变你原有逻辑）
# ----------------------------


def wrap_text(text, max_chars_per_line=18):
    """
    按字数进行简单的自动换行，兼容中文。
    可根据实际字体和宽度微调 max_chars_per_line。
    """
    text = text.replace("\n", " ").strip()
    lines = []
    while len(text) > max_chars_per_line:
        lines.append(text[:max_chars_per_line])
        text = text[max_chars_per_line:]
    lines.append(text)
    return "\n".join(lines)

# ---- 主函数 ----
def process_single_wav(audio_path, image_path):

    wav_path = Path(audio_path)

    print(f'\nstart process file [{audio_path}] ...')
    output_path = str(wav_path.with_suffix('.mp4'))
    srt_path = str(wav_path.with_suffix('.srt'))

    if not wav_path.exists():
        raise Exception(f'{str(wav_path)} not exists')

    if not os.path.exists(srt_path):
        raise Exception(f'{srt_path} not exists')

    if os.path.exists(output_path):
        print(f'output file exists [{output_path}], not process')
        return
    audio = AudioFileClip(audio_path)
    duration = audio.duration
    subs = pysrt.open(srt_path)

    # 背景
    video_w, video_h = 1280, 720
    bg_color = (245, 240, 230)  # 米色背景
    bg = ColorClip(size=(video_w, video_h), color=bg_color).with_duration(duration)

    # 图片区域（左边 10%-50%）
    img_clip = (
        ImageClip(image_path)
        .resized(height=int(video_h * 0.9))
        .with_position((int(video_w * 0.1), "center"))
        .with_duration(duration)
    )

    # 字幕区域（右边 50%-90%）
    text_area_w = int(video_w * 0.45)  # 宽度为总宽度的 40%
    text_x_pos = int(video_w * 0.5)   # 从 50% 开始
    max_chars_per_line = 14

    text_clips = []

    pre_end_time = 0
    for sub in subs:
        txt = sub.text.replace("\n", " ")
        start = sub.start.ordinal / 1000

        if pre_end_time !=0 and start < pre_end_time:
            start = pre_end_time
        end = sub.end.ordinal / 1000
        clip_duration = max(0.05, end - start)

        txt = wrap_text(sub.text, max_chars_per_line=max_chars_per_line)
        # 创建字幕片段
        try:
            text_clip = (
                TextClip(
                    text=txt + "\n",
                    font=FONT_PATH,
                    color="black",
                    font_size=36,
                    size=(text_area_w, 2000),  # 控制最大宽度，实现自动换行
                    # method="label",
                    method="caption",
                )
                .with_start(start)
                .with_duration(clip_duration)
                .with_position((text_x_pos, "center"))
            )
        except Exception as e:
            print(f"[TextClip ERROR] {audio_path} | text: {txt} | error: {e}")
            raise Exception(e)
        text_clips.append(text_clip)
        pre_end_time = end

    final = (
        CompositeVideoClip([bg, img_clip] + text_clips)
        .with_audio(audio)
        .with_duration(duration)
    )

    # 使用检测到或指定的编码器参数（仅影响编码器相关项）
    final.write_videofile(
        output_path,
        fps=24,
        codec=_CODEC_FOR_MOVIEPY,      # <- 可能是 "h264_nvenc" 或 "libx264"
        audio_codec="aac",
        ffmpeg_params=_FFMPEG_PARAMS_FOR_MOVIEPY,
        logger=None,
        threads=16,
    )

def combine_mp4(mp4_dir, mp4_regex):

    book_stem = Path(mp4_dir).stem

    output_path = os.path.join(mp4_dir, f"{book_stem}_all.mp4")

    list_file = os.path.join(mp4_dir, "mp4_list.txt")

    mp4_list = get_file_list_sorted(mp4_dir, mp4_regex)
    print(mp4_list)

    # 生成中间黑色视频文件（一次即可复用）
    black_clip = "/Users/larry/Documents/epub/black_0.5s.mp4"

    # 生成 0.5 秒黑色视频（静音）
    if not os.path.exists(black_clip):
        subprocess.run([
            "ffmpeg", "-f", "lavfi", "-i", "color=c=black:s=1920x1080:d=0.5",
            "-f", "lavfi", "-i", "anullsrc=r=44100:cl=stereo",
            "-shortest", "-c:v", "libx264", "-c:a", "aac", "-y", black_clip
        ])

    # 创建 concat 列表文件

    with open(list_file, "w") as f:
        for i, path in enumerate(mp4_list):
            f.write(f"file '{path}'\n")
            if i != len(mp4_list) - 1:  # 在中间插入 0.5 秒间隔
                f.write(f"file '{black_clip}'\n")

    # 拼接：将编码器参数插入 subprocess 调用（只修改编码器相关参数）
    # _CONCAT_PARAMS 已经是类似: ['-c:v','h264_nvenc','-preset','p4', ...] 或 cpu 版本
    concat_cmd = [
        "ffmpeg",
        "-f", "concat",
        "-safe", "0",
        # "-fflags", "+genpts",
        "-i", list_file,
    ]

    # 插入编码参数
    concat_cmd += _CONCAT_PARAMS

    # 常用输出参数（不改变原有行为）
    concat_cmd += [
        "-pix_fmt", "yuv420p",
        "-c:a", "aac",
        "-threads", "16",
        "-y", output_path
    ]

    subprocess.run(concat_cmd)

def get_file_list_sorted(wav_src_dir, wav_regex):
    pattern = re.compile(wav_regex)

    files = [
        f for f in os.listdir(wav_src_dir)
        if pattern.match(f)
    ]

    def sort_key(filename):
        m = pattern.match(filename)
        if m:
            second, first = map(int, m.groups())
            return (second, first)
        return (float('inf'), float('inf'))

    sorted_files = sorted(files, key=sort_key)

    sorted_paths = [os.path.join(wav_src_dir, f) for f in sorted_files]
    return sorted_paths

def process_dir(audio_dir, wav_regex, image_path):

    for s_path in get_file_list_sorted(audio_dir, wav_regex):
        process_single_wav(s_path, image_path)

# 测试moviepy的视频布局和效果
def test_font_preview1():
    """生成一个 10 秒测试视频，布局与 process_single_wav 完全一致。"""

    duration = 10
    video_w, video_h = 1280, 720

    # 背景
    bg_color = (245, 240, 230)
    bg = ColorClip(size=(video_w, video_h), color=bg_color).with_duration(duration)

    # ----------------------------
    # 左侧图片区域模拟（与 process_single_wav 对齐)
    # ----------------------------
    # 用浅灰色方块代替实际图片（避免依赖 image_path）
    fake_img = (
        ColorClip(size=(int(video_w * 0.38), int(video_h * 0.9)), color=(220, 220, 220))
        .with_position((int(video_w * 0.1), "center"))
        .with_duration(duration)
    )

    # ----------------------------
    # 字幕区域（右侧 50%-90%）——完全对齐主流程
    # ----------------------------
    text_area_w = int(video_w * 0.45)
    text_x_pos = int(video_w * 0.5)
    max_chars_per_line = 14

    #180字 ok
    txt = "此书标志着美国存在心理学本土化的完成的。此书标志着美国存在心理学本土化的完成的。此书标志着美国存在心理学本土化的完成的。此书标志着美国存在心理学本土化的完成的。此书标志着美国存在心理学本土化的完成的。此书标志着美国存在心理学本土化的完成的。此书标志着美国存在心理学本土化的完成的。此书标志着美国存在心理学本土化的完成的。此书标志着美国存在心理学本土化的完成的。"

    txt = wrap_text(txt, max_chars_per_line=max_chars_per_line)

    text_clip = (
        TextClip(
            text=txt + "\n",
            font=FONT_PATH,
            color="black",
            font_size=36,
            size=(text_area_w, None),
            method="label",        # 与正式流程一致
        )
        .with_position((text_x_pos, "center"))
        .with_duration(duration)
    )

    final = CompositeVideoClip([bg, fake_img, text_clip])

    final.write_videofile(
        "font_test.mp4",
        fps=24,
        codec=_CODEC_FOR_MOVIEPY
    )

# 测试滚动字幕
def test_font_preview(IMAGE_PATH, SRT_PATH):
    """按句滚动、多行显示、高亮当前字幕（稳定版）"""
    video_w, video_h = 1280, 720
    text_area_w = int(video_w * 0.45)
    text_x_pos = int(video_w * 0.5)
    lines_per_block = 6
    font_size = 36
    line_height = font_size + 10

    # 背景和图片
    bg = ColorClip(size=(video_w, video_h), color=(245, 240, 230))
    img_clip = (
        ImageClip(IMAGE_PATH)
        .resized(height=int(video_h * 0.9))
        .with_position((int(video_w * 0.1), "center"))
    )

    # 读取字幕
    subs = pysrt.open(SRT_PATH)
    all_lines = []
    for sub in subs:
        wrapped = wrap_text(sub.text, max_chars_per_line=14).split("\n")
        all_lines.extend(wrapped)
    total_lines = len(all_lines)

    text_clips = []

    for i, sub in enumerate(subs):
        start = sub.start.ordinal / 1000
        end = sub.end.ordinal / 1000
        duration = end - start

        wrapped_lines = wrap_text(sub.text, max_chars_per_line=14).split("\n")
        start_idx = sum(len(wrap_text(s.text, max_chars_per_line=14).split("\n")) for s in subs[:i])
        end_idx = start_idx + len(wrapped_lines)

        block_start = max(0, start_idx - lines_per_block // 2)
        block_end = min(total_lines, block_start + lines_per_block)
        visible_lines = all_lines[block_start:block_end]

        # 补空行保证高度固定
        while len(visible_lines) < lines_per_block:
            visible_lines.append(" ")

        # 基础文本（黑色）
        base_txt = "\n".join(visible_lines)
        block_height = lines_per_block * line_height
        base_clip = TextClip(text=base_txt, font=FONT_PATH, font_size=font_size,
                             color="black", size=(text_area_w, block_height), method="label")
        base_clip = base_clip.with_start(start).with_duration(duration).with_position((text_x_pos, video_h//2 - block_height//2))

        # 高亮文本（黄色）
        hl_lines = [""] * lines_per_block
        for idx in range(len(visible_lines)):
            if start_idx - block_start <= idx < end_idx - block_start:
                hl_lines[idx] = visible_lines[idx]
        hl_txt = "\n".join(hl_lines)
        hl_clip = TextClip(text=hl_txt, font=FONT_PATH, font_size=font_size,
                           color="yellow", size=(text_area_w, block_height), method="label")
        hl_clip = hl_clip.with_start(start).with_duration(duration).with_position((text_x_pos, video_h//2 - block_height//2))

        text_clips.extend([base_clip, hl_clip])

    final_duration = subs[-1].end.ordinal / 1000
    bg = bg.with_duration(final_duration)
    img_clip = img_clip.with_duration(final_duration)
    final = CompositeVideoClip([bg, img_clip] + text_clips).with_duration(final_duration)

    final.write_videofile("data_out/font_test_subtitle_slide_fixed.mp4", fps=24, codec=_CODEC_FOR_MOVIEPY, ffmpeg_params=_FFMPEG_PARAMS_FOR_MOVIEPY)

def run_start():

    # book_name = "fubaiyufanfu"
    book_name = "tianchaoyaoyuan"

    # test_font_preview()

    # test_font_preview("/Users/larry/Documents/epub/quanliyuwuzhi.jpg", "/Users/larry/github.com/tardigrade-dot/colab-script2/data_src/quanliyuwuzhi-0_0.srt")

    # combine_mp4(f"/Users/larry/Documents/epub/{book_name}", r'quanliyuwuzhi-(\d+)_(\d+)\.mp4')

    # process_dir("/Volumes/sw/MyDrive/xuese2/output", r'xuese2-(\d+)_(\d+)\.wav', "/Users/larry/github.com/tardigrade-dot/SenseVoice/data-forcedaligner/cover.jpeg")

    process_dir("/content/drive/MyDrive/tianchaoyaoyuan1", r'tianchaoyaoyuan1-(\d+)_(\d+)\.wav', "/content/drive/MyDrive/data_src/tianchaoyaoyuan_cover_page.png")

    # process_dir("/Users/larry/Documents/epub/fubaiyufanfu", r'fubaiyufanfu-(\d+)_(\d+)\.wav', "/Users/larry/Documents/epub/fubaiyufanfu-cover.jpg")

    # process_dir("/Users/larry/Documents/epub/quanliyuwuzhi", r'quanliyuwuzhi-(\d+)_(\d+)\.wav', "/Users/larry/Documents/epub/quanliyuwuzhi.jpg")

    # process_single_wav("/Users/larry/Documents/epub/quanliyuwuzhi/quanliyuwuzhi-2_1.wav", "/Users/larry/Documents/epub/quanliyuwuzhi.jpg")

run_start()